# Getting started with SQLAlchemy

In [1]:
import sqlalchemy
sqlalchemy.__version__


ModuleNotFoundError: No module named 'sqlalchemy'

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [4]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2024-07-11 16:03:52,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:52,665 INFO sqlalchemy.engine.Engine select 'hello world'
2024-07-11 16:03:52,667 INFO sqlalchemy.engine.Engine [generated in 0.00276s] ()
[('hello world',)]
2024-07-11 16:03:52,669 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"), 
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    conn.commit()


2024-07-11 16:03:52,682 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:52,684 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2024-07-11 16:03:52,686 INFO sqlalchemy.engine.Engine [generated in 0.00321s] ()
2024-07-11 16:03:52,688 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-07-11 16:03:52,790 INFO sqlalchemy.engine.Engine [generated in 0.10298s] [(1, 1), (2, 4)]
2024-07-11 16:03:52,793 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
with engine.connect() as conn:
    result = conn.execute(text("select * from some_table"))
    print(result.all())

2024-07-11 16:03:52,802 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:52,803 INFO sqlalchemy.engine.Engine select * from some_table
2024-07-11 16:03:52,805 INFO sqlalchemy.engine.Engine [generated in 0.00332s] ()
[(1, 1), (2, 4)]
2024-07-11 16:03:52,807 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"), 
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )
    

2024-07-11 16:03:52,813 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:52,815 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2024-07-11 16:03:52,817 INFO sqlalchemy.engine.Engine [cached since 0.1293s ago] [(6, 8), (9, 10)]
2024-07-11 16:03:52,818 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
with engine.connect() as conn:
    result = conn.execute(text("select * from some_table"))
    print(result.all())

2024-07-11 16:03:52,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:52,870 INFO sqlalchemy.engine.Engine select * from some_table
2024-07-11 16:03:52,872 INFO sqlalchemy.engine.Engine [cached since 0.07053s ago] ()
[(1, 1), (2, 4), (6, 8), (9, 10)]
2024-07-11 16:03:52,875 INFO sqlalchemy.engine.Engine ROLLBACK


## Statements execution

### Fetching rows

In [9]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-07-11 16:03:52,903 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:52,905 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2024-07-11 16:03:52,908 INFO sqlalchemy.engine.Engine [generated in 0.00483s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2024-07-11 16:03:52,910 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :myparam"), {"myparam": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-07-11 16:03:53,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:53,066 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2024-07-11 16:03:53,096 INFO sqlalchemy.engine.Engine [generated in 0.04364s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2024-07-11 16:03:53,099 INFO sqlalchemy.engine.Engine ROLLBACK


### Executing with an ORM Session


In [11]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :ymin ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"ymin": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2024-07-11 16:03:53,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-11 16:03:53,155 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2024-07-11 16:03:53,158 INFO sqlalchemy.engine.Engine [generated in 0.00258s] (6,)
x: 6  y: 8
x: 9  y: 10
2024-07-11 16:03:53,161 INFO sqlalchemy.engine.Engine ROLLBACK
